# ilastikProcessing
To effectively use ilastik some formating must be done before and after ilastik processes the images. The input is assumed to be a time-series of images stored in a multi-page TIFF.

In [2]:
%matplotlib inline
import glob
import h5py
import matplotlib
import numpy
import os
import os.path
import pandas
import re
import skimage
import skimage.exposure
import skimage.io
import skimage.transform
import subprocess
from tqdm._tqdm_notebook import tqdm_notebook

# Unpack (or separate) input images into single files.

## Update input image variables
Update the variable *image_directory* with the path to a folder that contains the input images. Update the *regex_image* variable to process only the images that match the regular expression. If the *regex_image* variable is equal to `(.*)\.tif`, then any *.tif* in the folder will be processed.

If a filename matches the *regex_single* regular expression, then it is assumed that this image has already been unpacked. An unpacked single image will have timepoint appended to the end of the file following the pattern `\-\d{4}\.tif`

*path_to_ilastik* is a string with the path to the ilastik software for [running headless](http://ilastik.org/documentation/basics/headless.html).

In [9]:
image_directory = r"C:\Users\karhohs\Documents\GitHub\2017_06_20_Machlus_megakarocyte_differentiation\strategy_ilastik_downsample\images"
path_to_ilastik = r"C:\Program Files\ilastik-1.2.2rc4\run-ilastik.bat"
path_to_project = r"C:\Users\karhohs\Documents\GitHub\2017_06_20_Machlus_megakarocyte_differentiation\strategy_ilastik_downsample\ilastik\Mouse_MK_training_Dec_2017.ilp"
path_to_cellprofiler = r"C:\Program Files (x86)\CellProfiler_3_0_0\CellProfiler.exe"
path_to_cp_pipeline = r"C:\Users\karhohs\Documents\GitHub\2017_06_20_Machlus_megakarocyte_differentiation\strategy_ilastik\pipelines\quantify_ilastik_no_model_cp3.cppipe"
regex_image = "(.*)\.tif"
regex_single = ".*\-\d{4}\.tif"
re_image = re.compile(regex_image)
re_single = re.compile(regex_single)

In [4]:
os.makedirs(os.path.join(image_directory, "single_images"), exist_ok=True)
imdir_single = os.path.join(image_directory,"single_images")
os.makedirs(os.path.join(image_directory, "ilastik"), exist_ok=True)
imdir_ilastik = os.path.join(image_directory,"ilastik")
os.makedirs(os.path.join(image_directory, "output"), exist_ok=True)
output_directory = os.path.join(image_directory,"output")

## Methods to import image metadata
* *is_my_file* will use the regular expression to filter image files to be processed.
* *make_dict* parses a file to be processed and places metadata into a dictionary.

Parse the files to be processed and then place the metadata into a Pandas dataframe.

In [5]:
def is_my_file(filename, re_image, re_single):
    
    mybool = False
    
    if (    re_image.match(filename) != None 
        and re_single.match(filename) == None
       ):
        
        mybool = True
        
    return mybool


def make_dict(filename, path, re_obj):
    
    my_dict = re_obj.match(filename).groupdict()
    
    my_dict["filename"] = filename
    
    my_dict["path"] = path
    
    return my_dict

In [6]:
image_files_dict = [make_dict(f, image_directory, re_image) for f in os.listdir(image_directory) if is_my_file(f, re_image, re_single)]
image_df = pandas.DataFrame(image_files_dict)

## Unpack the multi-page TIFF images
For every image, create a single image file for each timepoint. The input images are assumed to be RGB, which has 3 dimensions (length, width, color). The multipage TIFF of RGB images will have 4 dimensions (timepoints, length, width, color). 

*If the upstream workflow changes and the input image format is altered, then the conditional logic below will need to be updated, specifically the logic based on the shape of the input images.*

### Are the images across experiments similar enough to treat equally
One concern is that overfitting from training a classification model either through ilastik or CellProfiler analyst. The training set needs to be representative of the possibility space. This is accomplished by choosing a large enough image set that includes images of all states of interest including undifferentiated and fully differentiated megakaryocytes.

We also want to eliminate noise from known sources of variablity that could potentially weaken the classifier. The primary sources of noise in the images will be non-uniform illumination and differences in exposure. Non-uniform illumination is difficult to correct, because the background is actually in the middle of the intensity range and the signal occupies both high and low intensities.



In [17]:
#filelist = glob.glob(r"C:\Users\karhohs\Documents\GitHub\2017_06_20_Machlus_megakarocyte_differentiation\strategy_ilastik_downsample\images\single_images\*.tif")

#for f in filelist:
    
#    im = skimage.io.imread(f)
    
#    im2 = skimage.transform.rescale(im, 0.5)
        
#    im2 = skimage.color.rgb2gray(im2)
        
#    im2 = skimage.img_as_ubyte(im2)

#    skimage.io.imsave(f, im2)

In [18]:
def df_stack_image(p):
    
    im = skimage.io.imread(os.path.join(p["path"], p["filename"]))
    
    if len(im.shape) < 4:
        
        retest = re_image.match(p["filename"])

        retest.group(1)

        fname = "{0}-{1:04d}.tif".format(retest.group(1), 0)
        
        im2 = skimage.transform.rescale(im, 0.5)
        
        im2 = skimage.color.rgb2gray(im2)
        
        im2 = skimage.img_as_ubyte(im2)
        
        skimage.io.imsave(os.path.join(p["path"], fname), im2)
        
    else:
    
        number_of_timepoints = im.shape[0]

        for i in range(number_of_timepoints):

            retest = re_image.match(p["filename"])

            retest.group(1)

            fname = "{0}-{1:04d}.tif".format(retest.group(1), i)
            
            im2 = skimage.transform.rescale(im[i,:,:,:], 0.5)
            
            im2 = skimage.color.rgb2gray(im2)
        
            im2 = skimage.img_as_ubyte(im2)
            
            im2 = skimage.transform.rescale(im2, 0.5)

            skimage.io.imsave(os.path.join(p["path"], "single_images", fname), im2)

In [10]:
if image_df.empty is False:
    
    # Note that this can fail if the input images aren't in the expected format
    # If you receive an error, double check the format of the input images, e.g. are they RGB?
    tqdm_notebook.pandas(desc="unpack")
    _ = image_df.progress_apply(df_stack_image, axis=1)

else:
    
    print("no images to unpack")

no images to unpack


# Run ilastik

Using the single images created earlier, process the images using ilastik. First, create another dataframe with the single image metadata. Note, this has been written for running on Windows.

## Process ilastik output for CellProfiler
ilastik will output and HDF5 file that must be parsed for use as input to CellProfiler. This workflow assumes the default export settings are being used in ilastik. We have observed performance costs when changing the exporting settings to formats beyond the standard ilastik HDF5 file. For example, exporting TIFF images changes the shape of the exported data from yxc (the default) to cyx. This rearrangement will cause downstream errors, because the code as written expects the channel to be the third dimension.

### ilastik stage-2 labels
The project file *Mouse_MK.ilp* has the following labels that are stored in the same order within the HDF5 output.
1. background
1. border_white
1. cell
1. protrusion
1. background_border
1. not_cell

In [13]:
def is_my_file(filename, re_obj):
    
    mybool = False
    
    if re_obj.match(filename) != None:
        
        mybool = True
        
    return mybool


def make_dict(filename, path, re_obj):
    
    my_dict = re_obj.match(filename).groupdict()
    
    my_dict["filename"] = filename
    
    my_dict["path"] = path
    
    return my_dict

In [15]:
image_files_dict = [make_dict(f, imdir_single, re_single) for f in os.listdir(imdir_single) if is_my_file(f, re_single)]
image_df = pandas.DataFrame(image_files_dict)

In [16]:
def df_ilastik(p):
    
    filename = os.path.join(p["path"], p["filename"])
    
    filename_noext = os.path.splitext(p["filename"])[0]
    
    filename_h5 = "{}_Probabilities Stage 2.h5".format(filename_noext)
    
    # Run ilastik using subprocess
    
    process = subprocess.Popen([path_to_ilastik, 
                  "--headless",
                  "--export_source=probabilities stage 2",
                  "--output_format=hdf5",
                  r"--project={}".format(path_to_project),
                  filename
                 ], stdout=subprocess.PIPE)
    
    out, err = process.communicate()
    
    # unpack the HDF5 file
    
    label_list = ["background", "protrusion", "cell_boundary", "cell"]
    
    path_h5 = os.path.join(p["path"], filename_h5)
    
    with h5py.File(path_h5, "r") as ilastik_hdf5:
    
        ilastik_probabilities = ilastik_hdf5["exported_data"].value
    
        for i in range(ilastik_probabilities.shape[2]):
            im = skimage.img_as_uint(ilastik_probabilities[:, :, i])
        
            filename_slice = "{}_{}_prbstg2_{}.png".format(filename_noext, label_list[i], i)
        
            skimage.io.imsave(os.path.join(p["path"], "..", "ilastik", filename_slice), im)
    
    os.remove(path_h5)

In [ ]:
tqdm_notebook.pandas(desc="run ilastik")
_ = image_df.progress_apply(df_ilastik, axis=1)

HBox(children=(IntProgress(value=0, description='run ilastik', max=10), HTML(value='')))

C:\Users\karhohs\AppData\Local\Continuum\Miniconda3\envs\bioimg\lib\site-packages\skimage\util\dtype.py:122: UserWarning: Possible precision loss when converting from float32 to uint16
  .format(dtypeobj_in, dtypeobj_out))


# Run CellProfiler

## Make a filelist
Add the paths to each file that will be processed by CellProfiler into a text file.

## Run CellProfiler
Use subprocess again to run CellProfiler on the images to be processed.

Note, that a model that filters protrusions was trained in CellProfiler Analyst outside of this workflow. The model has to be in the input folder to be found by CellProfiler.

In [5]:
single_list = glob.glob(os.path.join(imdir_single,"*.tif"))
ilastik_list = glob.glob(os.path.join(imdir_ilastik,"*.png"))
big_list = single_list + ilastik_list
with open(os.path.join(image_directory,"filelist.txt"), 'w') as f:
    for item in big_list:
        f.write("{}\n".format(item))

In [16]:
process = subprocess.Popen([path_to_cellprofiler,
                  "--run-headless",
                  "--pipeline={}".format(path_to_cp_pipeline),
                  "--file-list={}".format(os.path.join(image_directory,"filelist.txt")),
                  "--image-directory={}".format(image_directory),
                  "--output-directory={}".format(output_directory)
                 ], stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True)
    
out, err = process.communicate()

In [17]:
print(out)

b''


In [18]:
print(err)

b'Times reported are CPU and Wall-clock times for each module\r\nFri Dec  1 19:07:32 2017: Image # 1, module Images # 1: CPU_time = 0.00 secs, Wall_time = 0.00 secs\r\nFri Dec  1 19:07:32 2017: Image # 1, module Metadata # 2: CPU_time = 0.00 secs, Wall_time = 0.00 secs\r\nFri Dec  1 19:07:32 2017: Image # 1, module NamesAndTypes # 3: CPU_time = 0.52 secs, Wall_time = 0.22 secs\r\nFri Dec  1 19:07:32 2017: Image # 1, module Groups # 4: CPU_time = 0.00 secs, Wall_time = 0.00 secs\r\nFri Dec  1 19:07:32 2017: Image # 1, module MeasureImageIntensity # 5: CPU_time = 0.27 secs, Wall_time = 0.29 secs\r\nFri Dec  1 19:07:33 2017: Image # 2, module Images # 1: CPU_time = 0.00 secs, Wall_time = 0.00 secs\r\nFri Dec  1 19:07:33 2017: Image # 2, module Metadata # 2: CPU_time = 0.00 secs, Wall_time = 0.00 secs\r\nFri Dec  1 19:07:33 2017: Image # 2, module NamesAndTypes # 3: CPU_time = 0.34 secs, Wall_time = 0.11 secs\r\nFri Dec  1 19:07:33 2017: Image # 2, module Groups # 4: CPU_time = 0.00 secs, 